In [ ]:
!conda install gdal -y

In [58]:
from typing import Dict, Generator, List
from osgeo import gdal, ogr, osr

# Enable GDAL/OGR exceptions
gdal.UseExceptions()
gdal.SetConfigOption('OGR_WFS_LOAD_MULTIPLE_LAYER_DEFN', 'NO')

# A mapping from ogr types to jsonschema types
OGR_DATA_TYPES = {
    0: ['null', 'integer'],
    2: ['null', 'number'],
    4: ['null', 'string'],
    12: ['null', 'integer']
}


class GeoLayer:
    def __init__(self, layer: ogr.Layer, source_srid: int, target_srid: int) -> None:
        self.layer: ogr.Layer = layer
        self.source_srid: int = source_srid
        self.target_srid: int = target_srid
        self.should_transform: bool = self.source_is_target == False
        self.transformer = self.create_transformer()

    @property
    def name(self) -> str:
        """Get the name of the layer, we need to replace the ':' as it interferes with the bigquery table names 

        Returns:
            str: The name of the layer
        """
        return self.layer.GetName().replace(':', '_').lower()

    @property
    def has_geometry(self) -> bool:
        """Check if the layer has geometry

        Returns:
            bool: Returns true if the layer has geometry
        """
        return self.layer.GetGeomType() != 0

    @property
    def source_projection(self) -> osr.SpatialReference:
        """The original reference system from the data source

        Returns:
            osr.SpatialReference: The original spatial reference
        """
        if self.source_srid:
            source_reference = osr.SpatialReference()
            source_reference.ImportFromEPSG(self.source_srid)
        else:
            source_reference = self.layer.GetSpatialRef()

        # If there source reference system cannot be found
        if not source_reference:
            raise Exception(
                'Source spatial reference system cannot be infered, please supply it using the config'
            )

        return source_reference

    @property
    def target_projection(self) -> osr.SpatialReference:
        """The target reference system, defined by the user

        Returns:
            osr.SpatialReference: The target spatial reference
        """
        target_reference = osr.SpatialReference()
        target_reference.ImportFromEPSG(self.target_srid)
        return target_reference

    @property
    def source_is_target(self) -> bool:
        """Check if the source projection is the same as the target
        projection.

        Returns:
            bool: True, if source projection is equal to the target projection
        """
        if not self.target_srid:
            return True
        else:
            is_same = self.source_projection.IsSame(self.target_projection)
            return is_same == 1

    def create_transformer(self) -> osr.CoordinateTransformation:
        """Creates a transformer to re-project the ingested geometries

        Returns:
            osr.CoordinateTransformation: The spatial transformer object
        """
        return osr.CoordinateTransformation(self.source_projection, self.target_projection)

    @property
    def schema(self) -> Dict[str, Dict[str, List[str]]]:
        layer_schema = {}

        # Loop through the fields in the schema
        for field in self.layer.schema:
            layer_schema[field.name] = {
                'type': OGR_DATA_TYPES.get(
                    field.type,
                    [
                        "null",
                        "string"
                    ]
                )
            }

        # Add a default geometry field if the layer has geometry
        if self.has_geometry:
            layer_schema['geometry'] = {
                'format': 'geography',
                'type': OGR_DATA_TYPES[4]
            }

        return layer_schema

    def features(self) -> Generator[Dict, None, None]:
        # Iterate over features
        feature = self.layer.GetNextFeature()
        while feature is not None:

            # Create a record with all the non spatial data
            record = {
                **feature.items()
            }

            # Add geometry to record if the layer has geometry
            if self.has_geometry:
                geometry = feature.GetGeometryRef()

                # If a re-projection is defined
                if self.should_transform:
                    geometry.Transform(self.transformer)

                record['geometry'] = geometry.GetLinearGeometry().ExportToJson()

            # Yield the record
            yield record

            # Get the next feature
            feature = self.layer.GetNextFeature()


class GeoSource:
    def __init__(self, path: str, config: Dict) -> None:
        self.path: str = path
        self.data_source: ogr.DataSource = ogr.Open(path)
        self.config: Dict = config

    @property
    def layers(self) -> Dict[str, GeoLayer]:
        target_srid = self.config.get('target_srid', None)
        source_srid = self.config.get('source_srid', None)
        include_layers = self.config.get('include_layers', None)

        # Fetch all layers
        layers = [GeoLayer(layer=layer,
                           source_srid=source_srid,
                           target_srid=target_srid)
                  for layer
                  in self.data_source if layer]

        # If a selection of layers was made
        if include_layers:
            return {layer.name: layer for layer in layers if layer.name in include_layers}
        # Return all layers
        else:
            return {layer.name: layer for layer in layers}

In [59]:
geosource = GeoSource(
    path='WFS:https://geodata.nationaalgeoregister.nl/wijkenbuurten2020/wfs?request=GetCapabilities&version=2.0.0',
#     path='WFS:http://geo.leefbaarometer.nl/leefbaarometer/ows?service=WFS&version=1.0.0',
    config={
        'target_srid': 4326
    }
)

print(geosource.data_source)
geosource.layers

<osgeo.ogr.DataSource; proxy of <Swig Object of type 'OGRDataSourceShadow *' at 0x7f87a0989030> >


{'wijkenbuurten2020_cbs_buurten_2020': <__main__.GeoLayer at 0x7f87a092ba30>,
 'wijkenbuurten2020_cbs_wijken_2020': <__main__.GeoLayer at 0x7f87a092bbb0>,
 'wijkenbuurten2020_gemeenten2020': <__main__.GeoLayer at 0x7f87a092b9d0>}

In [60]:
layer = geosource.layers[list(geosource.layers.keys())[0]]

In [61]:
for field in layer.layer.schema:
    print(field.type)

4
4
4
4
4
4
4
0
4
4
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
2
0
0
0
0
0
0
0
0
0
0
4


In [62]:
feature = layer.layer.GetNextFeature()

In [63]:
feature.items()

{'gml_id': 'cbs_buurten_2020.1',
 'buurtcode': 'BU00109998',
 'buurtnaam': '',
 'wijkcode': 'WK001099',
 'wijknaam': '',
 'gemeentecode': 'GM0010',
 'gemeentenaam': 'Delfzijl',
 'indelingswijziging_wijken_en_buurten': -99999999,
 'water': 'JA',
 'meest_voorkomende_postcode': '',
 'dekkingspercentage': -99999999,
 'omgevingsadressendichtheid': -99999999,
 'stedelijkheid_adressen_per_km2': -99999999,
 'bevolkingsdichtheid_inwoners_per_km2': -99999999,
 'aantal_inwoners': -99999999,
 'mannen': -99999999,
 'vrouwen': -99999999,
 'percentage_personen_0_tot_15_jaar': -99999999,
 'percentage_personen_15_tot_25_jaar': -99999999,
 'percentage_personen_25_tot_45_jaar': -99999999,
 'percentage_personen_45_tot_65_jaar': -99999999,
 'percentage_personen_65_jaar_en_ouder': -99999999,
 'percentage_ongehuwd': -99999999,
 'percentage_gehuwd': -99999999,
 'percentage_gescheid': -99999999,
 'percentage_verweduwd': -99999999,
 'aantal_huishoudens': -99999999,
 'percentage_eenpersoonshuishoudens': -9999999

In [64]:
next(layer.features())

{'gml_id': 'cbs_buurten_2020.2',
 'buurtcode': 'BU00349997',
 'buurtnaam': '',
 'wijkcode': 'WK003499',
 'wijknaam': '',
 'gemeentecode': 'GM0034',
 'gemeentenaam': 'Almere',
 'indelingswijziging_wijken_en_buurten': -99999999,
 'water': 'JA',
 'meest_voorkomende_postcode': '',
 'dekkingspercentage': -99999999,
 'omgevingsadressendichtheid': -99999999,
 'stedelijkheid_adressen_per_km2': -99999999,
 'bevolkingsdichtheid_inwoners_per_km2': -99999999,
 'aantal_inwoners': -99999999,
 'mannen': -99999999,
 'vrouwen': -99999999,
 'percentage_personen_0_tot_15_jaar': -99999999,
 'percentage_personen_15_tot_25_jaar': -99999999,
 'percentage_personen_25_tot_45_jaar': -99999999,
 'percentage_personen_45_tot_65_jaar': -99999999,
 'percentage_personen_65_jaar_en_ouder': -99999999,
 'percentage_ongehuwd': -99999999,
 'percentage_gehuwd': -99999999,
 'percentage_gescheid': -99999999,
 'percentage_verweduwd': -99999999,
 'aantal_huishoudens': -99999999,
 'percentage_eenpersoonshuishoudens': -99999999,

In [53]:
layer.has_geometry

True

In [55]:
feature = layer.layer.GetNextFeature()
feature

<osgeo.ogr.Feature; proxy of <Swig Object of type 'OGRFeatureShadow *' at 0x7f87a096d270> >

In [56]:
geometry = feature.GetGeometryRef()
geometry

<osgeo.ogr.Geometry; proxy of <Swig Object of type 'OGRGeometryShadow *' at 0x7f87a0941090> >

In [ ]:
if self.should_transform:
    geometry.Transform(self.transformer)

In [57]:
geometry.GetLinearGeometry().ExportToJson()

'{ "type": "MultiPolygon", "coordinates": [ [ [ [ 154927.5348, 474979.289999999979045 ], [ 154931.236, 474976.6641 ], [ 154933.2163, 474979.2376 ], [ 154933.4401, 474979.528600000019651 ], [ 154933.980000000010477, 474980.230099999986123 ], [ 154934.5693, 474980.995999999984633 ], [ 154934.636999999987893, 474981.083999999973457 ], [ 154937.018000000010943, 474977.683100000023842 ], [ 154936.541300000011688, 474976.571899999980815 ], [ 154936.4003, 474976.243399999977555 ], [ 154936.2505, 474975.894099999975879 ], [ 154935.997, 474975.303000000014435 ], [ 154934.5305, 474973.396100000012666 ], [ 154934.511, 474973.370799999975134 ], [ 154932.6428, 474970.941600000020117 ], [ 154932.597, 474970.881999999983236 ], [ 154934.2903, 474968.17249999998603 ], [ 154937.351499999989755, 474963.273899999971036 ], [ 154937.7741, 474962.597799999988638 ], [ 154939.397999999986496, 474959.99910000001546 ], [ 154942.4178, 474960.5024 ], [ 154943.3074, 474960.650499999988824 ], [ 154943.479, 474960.67

In [ ]:
if layer.has_geometry:
    geometry = feature.GetGeometryRef()

    # If a re-projection is defined
    if self.should_transform:
        geometry.Transform(self.transformer)

    record['geometry'] = geometry.GetLinearGeometry().ExportToJson()

In [39]:
layer.layer.GetGeomType()

6

In [40]:
geometry = feature.GetGeometryRef()

In [41]:
geometry.GetGeometryName()

'MULTIPOLYGON'

In [42]:
geometry.GetLinearGeometry().ExportToJson()

'{ "type": "MultiPolygon", "coordinates": [ [ [ [ 122259.5323, 487908.287100000015926 ], [ 122286.6247, 487900.4987 ], [ 122392.287, 487876.886 ], [ 122420.352, 487871.006 ], [ 122500.999, 487854.109 ], [ 122505.593, 487853.147 ], [ 122504.699, 487846.891 ], [ 122503.966, 487841.757999999972526 ], [ 122503.402, 487837.81 ], [ 122503.27, 487836.881999999983236 ], [ 122503.56, 487829.911000000021886 ], [ 122503.91, 487828.314000000013039 ], [ 122504.757, 487824.454000000027008 ], [ 122505.523, 487820.965000000025611 ], [ 122506.172, 487818.00199999997858 ], [ 122505.3, 487809.869 ], [ 122503.85, 487799.994 ], [ 122509.302, 487784.168 ], [ 122509.654, 487783.147 ], [ 122511.5906, 487778.309499999973923 ], [ 122511.817, 487777.744 ], [ 122515.732, 487767.962999999988824 ], [ 122516.1139, 487767.0085 ], [ 122516.5174, 487766.0 ], [ 122519.337, 487758.953099999984261 ], [ 122519.715, 487757.897 ], [ 122520.7316, 487755.051700000010896 ], [ 122522.363, 487750.48499999998603 ], [ 122522.888, 4

In [43]:
geometry.ExportToJson()

'{ "type": "MultiPolygon", "coordinates": [ [ [ [ 122259.5323, 487908.287100000015926 ], [ 122286.6247, 487900.4987 ], [ 122392.287, 487876.886 ], [ 122420.352, 487871.006 ], [ 122500.999, 487854.109 ], [ 122505.593, 487853.147 ], [ 122504.699, 487846.891 ], [ 122503.966, 487841.757999999972526 ], [ 122503.402, 487837.81 ], [ 122503.27, 487836.881999999983236 ], [ 122503.56, 487829.911000000021886 ], [ 122503.91, 487828.314000000013039 ], [ 122504.757, 487824.454000000027008 ], [ 122505.523, 487820.965000000025611 ], [ 122506.172, 487818.00199999997858 ], [ 122505.3, 487809.869 ], [ 122503.85, 487799.994 ], [ 122509.302, 487784.168 ], [ 122509.654, 487783.147 ], [ 122511.5906, 487778.309499999973923 ], [ 122511.817, 487777.744 ], [ 122515.732, 487767.962999999988824 ], [ 122516.1139, 487767.0085 ], [ 122516.5174, 487766.0 ], [ 122519.337, 487758.953099999984261 ], [ 122519.715, 487757.897 ], [ 122520.7316, 487755.051700000010896 ], [ 122522.363, 487750.48499999998603 ], [ 122522.888, 4

In [22]:
geometry.ExportToJ()

In [21]:
geometry.ExportToWkt()

'MULTISURFACE (((150213.998 479503.726,150087.299 479382.379,150000.42 479461.258,150000.3546 479461.3174,150000.3663 479461.3273,150001.4563 479462.22,150004.7258 479464.8985,150133.7643 479570.6095,150133.7709 479570.6153,150136.935 479573.316,150138.3409 479574.5163,150139.7313 479575.7029,150141.675 479577.3624,150141.7275 479577.407,150142.305 479577.9,150142.698 479577.584,150142.9333 479577.395,150142.9508 479577.3809,150143.439 479576.988,150143.4808 479576.9545,150144.494 479576.14,150223.3808 479512.713,150213.998 479503.726)),((141872.969 483192.3988,141872.9781 483192.3985,141872.9841 483192.3985,141904.5239 483191.7934,141912.341 483191.6435,141915.4884 483191.5831,142003.0729 483189.1549,142003.0854 483189.1545,142003.096 483189.1538,142003.1019 483189.1536,142003.1106 483189.1533,142003.1196 483189.1526,142082.3583 483183.2428,142086.5243 483182.9319,142086.5309 483182.9313,142086.5498 483182.9296,142086.5693 483182.9275,142086.5765 483182.9266,142091.9089 483182.2458,14

In [148]:
%%timeit
next(layer.features())

743 µs ± 3.43 µs per loop (mean ± std. dev. of 7 runs, 1000 loops each)


In [149]:
next(layer.features())

{'gebruiksdoel': 'industriefunctie',
 'oppervlakte': 1190,
 'nummeraanduidingRef': '0005200000034206',
 'pandRef': '0005100000000031',
 'lvID': '.',
 'namespace': None,
 'lokaalID': None,
 'versie': None,
 'status': 'Verblijfsobject in gebruik (niet ingemeten)',
 'geconstateerd': 0,
 'documentDatum': '2008/02/15',
 'documentNummer': '20070146',
 'voorkomenIdentificatie': 1,
 'beginGeldigheid': '2008/02/15',
 'eindGeldigheid': '2014/08/26',
 'tijdstipRegistratie': '2010/12/13 18:07:31',
 'eindRegistratie': '2014/08/26 11:50:48',
 'tijdstipInactief': None,
 'tijdstipRegistratieLV': '2010/12/14 10:08:53.881',
 'tijdstipEindRegistratieLV': '2014/08/26 12:01:40.265',
 'tijdstipInactiefLV': None,
 'tijdstipNietBagLV': None,
 'geometry': '{ "type": "Point", "coordinates": [ 53.263260466544786, 6.620064314312219 ] }'}